# `qpub` for python projects

this notebook tests different conformations of python files, and configuring them with `flit, poetry and setuptools`.

## a single python script

In [1]:
import pytest
@pytest.fixture
def single_script(pytester):
    import nbformat, dgaf
    name = "sample_python_project"
    pytester.makefile(".py", **{name: """'a sample project'
__version__ = '0.0.1'
    
import pandas"""})
    project = dgaf.Project(pytester.path)
    yield project

In [2]:
def test_single_script(single_script):
    assert single_script.fs.files
    assert not single_script.fs.top_level
    assert single_script.get_name() == "sample_python_project"
    assert "nbval" not in single_script.get_test_requires()
    assert not single_script.get_url()
    assert single_script.get_requires() == ["pandas"]

In [3]:
def test_single_script_flit(pytester, single_script):
    # when there is no url we can initialize one in a few ways.
    pyproject = single_script/__import__("dgaf").PYPROJECT_TOML
    assert not pyproject.exists()
    single_script.to_flit()
    assert pyproject.exists()
    result = pytester.run(*"flit build".split())
    assert not result.ret

In [4]:
def test_single_script_setuptools(pytester, single_script):
    setup = single_script/__import__("dgaf").SETUP_CFG
    assert not setup.exists()
    single_script.to_setuptools()
    assert setup.exists()
    single_script.to_setup_py()
    result = pytester.run(*"python setup.py build --dry-run".split())
    assert not result.ret

## a project in a python directory

In [5]:
import pytest
@pytest.fixture
def directory(pytester):
    name = "sample_python_project"
    pytester.mkdir(name)
    pytester.makefile(".py", **{F"{name}/__init__.py": """'a sample project'
__version__ = '0.0.1'
    
import pandas"""})
    project = __import__("dgaf").Project(pytester.path)
    yield project

In [6]:
def test_directory(directory):
    import dgaf
    assert directory.fs.content
    assert directory.fs.top_level
    assert directory.get_name() == "sample_python_project"
    assert "nbval" not in directory.get_test_requires()
    assert not directory.get_url()
    assert directory.get_requires() == ["pandas"]    

In [7]:
def test_directory_flit(pytester, directory):
    # when there is no url we can initialize one in a few ways.
    import dgaf
    pyproject = directory/__import__("dgaf").PYPROJECT_TOML
    assert not pyproject.exists()
    directory.to_flit()
    assert pyproject.exists()
    assert not pytester.run(*"flit build".split()).ret

In [8]:
def test_directory_poetry(pytester, directory):
    # when there is no url we can initialize one in a few ways.
    import dgaf
    pyproject = directory/__import__("dgaf").PYPROJECT_TOML
    assert not pyproject.exists()
    directory.to_poetry()
    assert pyproject.exists()
    assert not pytester.run(*"poetry build".split()).ret

In [9]:
def test_directory_setuptools(pytester, directory):
    import dgaf
    setup = directory/__import__("dgaf").SETUP_CFG
    assert not setup.exists()
    directory.to_setuptools()
    assert setup.exists()
    directory.to_setup_py()
    assert not pytester.run(*"python setup.py build --dry-run".split()).ret

## a python project in `src` layout

In [10]:
import pytest
@pytest.fixture
def src(pytester):
    
    name = "sample_python_src_project"
    pytester.mkdir("src"), pytester.mkdir(F"src/{name}")
    pytester.makefile(".py", **{F"src/{name}/__init__.py": """'a sample project'
__version__ = '0.0.1'
    
import pandas"""})
    project = __import__("dgaf").Project(pytester.path)
    yield project

In [11]:
def test_src_directory_poetry(pytester, src):
    # when there is no url we can initialize one in a few ways.
    name = src.get_name()
    print(name)
    assert src.get_name() == "sample_python_src_project"
    src.to_poetry()
    assert not pytester.run(*"poetry build".split()).ret 
    assert not pytester.run(*"python -m pip install . --no-deps".split()).ret
    
    import importlib_metadata
    assert importlib_metadata.distribution("sample_python_src_project")
    assert __import__("sample_python_src_project")

    def test_src_directory_setuptools(pytester, src):
        ...